Consultar o valor do Dolar por dia com API

In [ ]:
import pandas as pd
import requests
from datetime import date
import duckdb

In [1]:
!pip install yfinance
import yfinance as yf


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\Anderson\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:

ini = "04-01-2025"
fim = date.today().strftime("%m-%d-%Y")
url = (
  "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/"
  f"CotacaoDolarPeriodo(dataInicial=@ini,dataFinalCotacao=@fim)?"
  f"@ini='{ini}'&@fim='{fim}'&$select=cotacaoCompra,dataHoraCotacao&$top=10000&$format=json"
)

data = requests.get(url, timeout=30).json()["value"]
df = pd.DataFrame(data)
df["data"] = pd.to_datetime(df["dataHoraCotacao"]).dt.date

# última cotação do dia (fechamento)
df_daily = df.sort_values("dataHoraCotacao").groupby("data").tail(1)
df_daily = df_daily[["data","cotacaoCompra"]].reset_index(drop=True)

# formata data para dd/mm/yyyy
df_daily["data"] = pd.to_datetime(df_daily["data"]).dt.strftime("%d/%m/%Y")

In [ ]:
DB_PATH = "dados_dolar.duckdb"
con = duckdb.connect(DB_PATH)

# 1) cria tabela 
con.execute("""
CREATE TABLE IF NOT EXISTS dolar_diario (
  data DATE,
  cotacao_compra DOUBLE
);
""")

# 2) registra o DataFrame atual como uma view temporária
con.register("df_stage", df_daily)

# 3) upsert por data 
con.execute("""
MERGE INTO dolar_diario AS t
USING (
  SELECT
    STRPTIME(data, '%d/%m/%Y')::DATE AS data,
    CAST(cotacaoCompra AS DOUBLE)     AS cotacao_compra
  FROM df_stage
) AS s
ON t.data = s.data
WHEN MATCHED THEN UPDATE SET
  cotacao_compra = s.cotacao_compra
WHEN NOT MATCHED THEN INSERT (data, cotacao_compra)
VALUES (s.data, s.cotacao_compra);
""")

# 4) checagem
print(con.execute("""
SELECT *
FROM dolar_diario
ORDER BY data DESC
LIMIT 10;
""").fetchdf())

con.execute("""
COPY dolar_diario TO './exports/dolar_diario.csv' (HEADER, DELIMITER ',');
""")

con.close()


In [ ]:
# ================================
# Configurações
# ================================
DB_PATH = "dados_dolar.duckdb"
TABELA = "dolar_ohlc"
ANO = date.today().year

inicio = pd.Timestamp(ANO, 4, 1)
fim_desejado = pd.Timestamp(ANO, 10, 31)
fim = min(fim_desejado, pd.Timestamp.today().normalize())

ticker = "USDBRL=X"  # dólar/real no Yahoo Finance

# ================================
# 1) Baixar OHLC (mercado) do Yahoo
# ================================
df = yf.download(
  ticker,
  start=inicio,
  end=fim + pd.Timedelta(days=1), # inclui o último dia
  interval="1d",
  auto_adjust=False,
  progress=False,
)

if df.empty:
  raise SystemExit("Nenhum dado retornado pelo Yahoo Finance para o período.")

ohlc = (
  df.reset_index()[["Date", "Open", "High", "Low", "Close"]]
   .rename(columns={
     "Date": "data",
     "Open": "abertura",
     "High": "alta",
     "Low": "baixa",
     "Close": "fechamento",
   })
)

ohlc[["abertura","fechamento","alta","baixa"]] = (
  ohlc[["abertura","fechamento","alta","baixa"]].round(4)
)

print("Prévia OHLC:")
print(ohlc.head())

# ================================
# 2) Salvar no DuckDB
# ================================
con = duckdb.connect(DB_PATH)


con.execute(f"""
CREATE TABLE IF NOT EXISTS {TABELA} (
 data DATE PRIMARY KEY,
 abertura  DOUBLE,
 fechamento DOUBLE,
 alta    DOUBLE,
 baixa   DOUBLE
);
""")


con.register("df_stage", ohlc)
con.execute("CREATE OR REPLACE TEMP TABLE stage AS SELECT * FROM df_stage;")

con.execute("""
COPY dolar_ohlc TO './exports/dolar_ohlc.csv' (HEADER, DELIMITER ',');
""")

con.close()


ohlc.to_csv('./exports/dolar_ohlc_yahoo.csv', index=False)

In [ ]:
print("Resumo – Séries (PTAX & Yahoo)")
print("-------------------------------")
print(f"PTAX diário: {len(df_daily)} | Janela: {min_ptax} → {max_ptax}")
print(f"OHLC linhas: {len(ohlc)} | Janela: {min_ohlc} → {max_ohlc}")
print("Carga: dolar_diario & dolar_ohlc (DuckDB) + exports CSV em ./exports")